In [1]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs.utils import prepare_fashion, prepare_mnist
from libs.constants import MODELS_FOLDER

In [2]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
from libs.model_archs import lenetA, fcA

In [4]:
from libs.seeds import load_model_seeds
model_seeds = load_model_seeds()

In [5]:
# define dataset and model architecture
# dataset = "fashion"
dataset = "mnist"
model_arch = "lenetA"
# model_arch = "fcA"

# set training hyperparameters
batch_size = 32
n_epochs = 10

In [6]:
# global seed
seed = model_seeds[0]
tf.random.set_seed(seed)
np.random.seed(seed)

# prepare data
if dataset == "fashion":
    (x_train, y_train), (x_test, y_test) = prepare_fashion()
elif dataset == "mnist":
    (x_train, y_train), (x_test, y_test) = prepare_mnist()
else:
    print("Invalid Dataset or Dataset not found")

# create model
input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
n_classes = y_train.shape[1]

modelarchfuncList = {'lenetA': lenetA, 'fcA': fcA }
parameters = {'input_shape':input_shape, 'nb_classes':n_classes}

model = modelarchfuncList[model_arch](**parameters)

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'],
)

# train model
model.fit(x_train, 
          y_train, 
          batch_size=batch_size, 
          epochs=n_epochs,
          validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 17s 8ms/step - loss: 0.1813 - accuracy: 0.9452 - val_loss: 0.0772 - val_accuracy: 0.9778
Epoch 2/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0545 - accuracy: 0.9835 - val_loss: 0.0487 - val_accuracy: 0.9861
Epoch 3/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0392 - accuracy: 0.9884 - val_loss: 0.0472 - val_accuracy: 0.9858
Epoch 4/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0290 - accuracy: 0.9908 - val_loss: 0.0444 - val_accuracy: 0.9870
Epoch 5/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0230 - accuracy: 0.9927 - val_loss: 0.0494 - val_accuracy: 0.9863
Epoch 6/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0172 - accuracy: 0.9944 - val_loss: 0.0408 - val_accuracy: 0.9898
Epoch 7/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0151 - accuracy: 0.9953 - val_loss: 0.0438 - val_ac

In [7]:
# evaluate model
score = model.evaluate(x_test, 
                       y_test, 
                       batch_size=batch_size)

print("Original Accuracy: ",score)

313/313 [==============================] - 2s 5ms/step - loss: 0.0333 - accuracy: 0.9909
Original Accuracy:  [0.03325751796364784, 0.9908999800682068]


In [8]:
# save model
model_type = dataset + "--" + model_arch
model_instance = model_type + "-" + str(seed)
model_filename = model_instance + ".h5"
model_subdir = pathlib.Path(MODELS_FOLDER / model.name)
pathlib.Path(model_subdir).mkdir(parents=True, exist_ok=True)
model_file = str(pathlib.Path(model_subdir/ model_filename))
model.save(model_file)
model.summary()

Model: "lenetA"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 23, 23, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 9, 9, 16)         0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 5, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 3000)              0    